In [89]:
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha

In [90]:
import boto3

In [91]:
sess = boto3.Session()

In [92]:
#sess.get_available_services()

In [93]:
sess.region_name

'ap-southeast-2'

In [94]:
client = sess.client("sagemaker-runtime", region_name=sess.region_name)

In [95]:
import json

In [96]:
# Single Observation
sample_one = ['Lord ganesa is powerful.']
sample_two = ['Siva and Parvathi are universal parents.']
request = {
    "instances": [
        # First instance.
        {
            "features": sample_one
        },
        {
            "features": sample_two
        }
    ]
}
print(json.dumps(request, indent=2))

{
  "instances": [
    {
      "features": [
        "Lord ganesa is powerful."
      ]
    },
    {
      "features": [
        "Siva and Parvathi are universal parents."
      ]
    }
  ]
}


In [97]:
import pickle
import os

In [98]:
class sparse_to_dense:
    def fit(self, X, y=None, **fit_params):
        return self
    def transform(self, X, y=None, **fit_params):
        return X.toarray()

In [101]:
def transform(data):
    data_copy = json.loads(json.dumps(data))
    data_pipeline = None
    if os.path.exists('data_pipeline_downloaded') and os.path.isfile('data_pipeline_downloaded'):
        with open('data_pipeline_downloaded', 'rb') as fh:
            data_pipeline = pickle.load(fh)
    else:
        pipeln_bucket = 'varenya-ml-sagemaker'
        pipeln_object = 'binary_text_classification/data_pipeline/data_pipeline'
        with open('data_pipeline_downloaded', 'wb') as fw:
            boto3.Session().resource('s3').Bucket(pipeln_bucket).Object(pipeln_object).download_fileobj(fw)
        with open('data_pipeline_downloaded', 'rb') as fh:
            data_pipeline = pickle.load(fh)
    return data_pipeline.transform(data_copy)

In [102]:
X_test = [instance['features'] for instance in request['instances']]
X_test = transform(X_test)
X_test

array([[-0.07029673, -0.03537746, -0.03537746, ..., -0.03537746,
        -0.06820728, -0.05721465],
       [-0.07029673, -0.03537746, -0.03537746, ..., -0.03537746,
        -0.06820728, -0.05721465]])

In [85]:
X_test.shape

(2, 1587)

In [65]:
from sagemaker.serializers import CSVSerializer

In [66]:
endpoint_name = "xgboost-aws-v1"
serializer = CSVSerializer()

In [86]:
result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body='\n'.join([','.join([str(ite) for ite in item]) for item in X_test.tolist()]).encode('utf-8'),
                       ContentType='text/csv')

In [87]:
result = result['Body'].read().decode('utf-8')

In [88]:
y_pred = [float(item) for item in result.split() if item != ""]
y_pred

[0.3724002242088318, 0.3724002242088318]

In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('test.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587
0,1.0,-0.070297,-0.035377,-0.035377,-0.035377,-0.035377,-0.059998,-0.035377,-0.070848,-0.035377,...,-0.035377,-0.035377,-0.035377,-0.047469,-0.060202,-0.035377,-0.035377,-0.035377,-0.068207,-0.057215
1,1.0,-0.070297,-0.035377,-0.035377,-0.035377,-0.035377,-0.059998,-0.035377,-0.070848,-0.035377,...,-0.035377,-0.035377,-0.035377,-0.047469,-0.060202,-0.035377,-0.035377,-0.035377,-0.068207,-0.057215
2,1.0,-0.070297,-0.035377,-0.035377,-0.035377,-0.035377,-0.059998,-0.035377,-0.070848,-0.035377,...,-0.035377,-0.035377,-0.035377,-0.047469,-0.060202,-0.035377,-0.035377,-0.035377,-0.068207,-0.057215
3,1.0,-0.070297,-0.035377,-0.035377,-0.035377,-0.035377,-0.059998,-0.035377,-0.070848,-0.035377,...,-0.035377,-0.035377,-0.035377,-0.047469,-0.060202,-0.035377,-0.035377,-0.035377,-0.068207,-0.057215
4,1.0,-0.070297,-0.035377,-0.035377,-0.035377,-0.035377,-0.059998,-0.035377,-0.070848,-0.035377,...,-0.035377,-0.035377,-0.035377,-0.047469,-0.060202,-0.035377,-0.035377,-0.035377,-0.068207,-0.057215


In [48]:
X_test = df.iloc[:, 1:]
X_test.shape

(200, 1587)

In [56]:
y_pred = []
for i in range(0, len(X_test), 30):
    result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=serializer.serialize(X_test[i:i+30]).encode('utf-8'),
                       ContentType='text/csv')
    result = result['Body'].read().decode('utf-8')
    y_pred.extend([float(item) for item in result.split() if item != ""])

In [59]:
len(y_pred)

200

In [60]:
sk = boto3.client("sagemaker")

In [62]:
sk.serializers

AttributeError: 'SageMaker' object has no attribute 'serializers'

In [75]:
tmp = np.array([[1,2,3],[4,5,6]])

In [76]:
tmp

array([[1, 2, 3],
       [4, 5, 6]])

In [77]:
serializer.serialize(tmp)

'1,2,3\n4,5,6'

In [84]:
'\n'.join([','.join([str(ite) for ite in item]) for item in tmp.tolist()])

'1,2,3\n4,5,6'